In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_table('./data/round1_ijcai_18_train_20180301.txt', sep=' ')
data.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4016,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0


In [3]:
shop_data = data[['shop_id', 'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 
                 'shop_score_delivery', 'shop_score_description', 'is_trade']]
shop_data.head()

,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
0,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
1,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
2,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
3,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
4,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0


In [4]:
def feature_group(data, feature, is_trade):
    if (is_trade != 1 and is_trade != 0):
        return None
    else:
        data = data[[feature, 'is_trade']]
        data_feature = data[data['is_trade'] == is_trade]
        data_feature = data_feature.reset_index(drop=True)
        data_feature_grouped = data_feature.groupby([feature]).count()
        data_feature_grouped = data_feature_grouped.reset_index()
        data_feature_grouped = data_feature_grouped.sort_values(['is_trade'], ascending=False)
    return data_feature_grouped

In [5]:
def feature_log_cal(data_feature_grouped, feature, is_trade):
    if (is_trade != 1 and is_trade != 0):
        return None
    else:
        new_feature = feature + '_' + str(is_trade)
        data_feature_grouped[new_feature] = np.log(data_feature_grouped['is_trade'] / len(data_feature_grouped))
#         # 0-1标准化
#         data_feature_grouped[new_feature] =  (data_feature_grouped['is_trade'] - data_feature_grouped['is_trade'].min())/\
#                                         (data_feature_grouped['is_trade'].max() - data_feature_grouped['is_trade'].min())
    return data_feature_grouped

# 1. shop_id

In [6]:
shop_id_grouped_1 = feature_group(shop_data, 'shop_id', 1)
shop_id_grouped_1 = feature_log_cal(shop_id_grouped_1, 'shop_id', 1)
shop_id_grouped_1.head()

,shop_id,is_trade,shop_id_1
285,1932086538588242822,255,-1.769287
1456,8953967780295706657,249,-1.793097
304,2025576994711098052,178,-2.128767
1439,8860698437199963449,110,-2.610070
674,4215742678121737516,102,-2.685577


In [7]:
shop_id_grouped_0 = feature_group(shop_data, 'shop_id', 0)
shop_id_grouped_0 = feature_log_cal(shop_id_grouped_0, 'shop_id', 0)
shop_id_grouped_0.head()

,shop_id,is_trade,shop_id_0
2794,6597981382309269962,11212,1.043269
786,1932086538588242822,9447,0.871982
3832,8953967780295706657,8104,0.718642
827,2025576994711098052,4994,0.234522
1746,4215742678121737516,4194,0.059939


In [8]:
shop_id_grouped_1.to_csv('./feature_data/shop_data/shop_id_grouped_1.txt', sep=' ', index=False, encoding='utf-8')
shop_id_grouped_0.to_csv('./feature_data/shop_data/shop_id_grouped_0.txt', sep=' ', index=False, encoding='utf-8')

# 2. shop_star_level（其余变量都为连续值变量）

In [9]:
shop_star_level_grouped_1 = feature_group(shop_data, 'shop_star_level', 1)
shop_star_level_grouped_1 = feature_log_cal(shop_star_level_grouped_1, 'shop_star_level', 1)
shop_star_level_grouped_1

,shop_star_level,is_trade,shop_star_level_1
13,5013,1835,4.470277
14,5014,1799,4.450464
15,5015,1356,4.167772
12,5012,1280,4.110093
11,5011,552,3.269026
17,5017,427,3.012262
10,5010,416,2.986163
16,5016,393,2.929287
9,5009,297,2.649210
18,5018,269,2.550189


In [10]:
shop_star_level_grouped_0 = feature_group(shop_data, 'shop_star_level', 0)
shop_star_level_grouped_0 = feature_log_cal(shop_star_level_grouped_0, 'shop_star_level', 0)
shop_star_level_grouped_0

,shop_star_level,is_trade,shop_star_level_0
14,5013,80806,8.208764
15,5014,79797,8.196199
16,5015,72636,8.102173
13,5012,72331,8.097966
12,5011,33804,7.337292
19,5018,24806,7.027798
11,5010,24071,6.997721
18,5017,22816,6.944175
17,5016,18846,6.753014
10,5009,17894,6.701178


In [11]:
shop_star_level_grouped_1.to_csv('./feature_data/shop_data/shop_star_level_grouped_1.txt', sep=' ', index=False, encoding='utf-8')
shop_star_level_grouped_0.to_csv('./feature_data/shop_data/shop_star_level_grouped_0.txt', sep=' ', index=False, encoding='utf-8')